In [214]:
import config
from bs4 import BeautifulSoup as BS
import requests
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import calinski_harabasz_score
from sklearn.decomposition import PCA
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

In [5]:
df_scaled2 = pd.read_csv('df_scaled2.csv', index_col='Team')

In [6]:
df_scaled2.head()

,ppda_coef,deep,S,CR,CCOP,CCSP,P,DR,IBS,OBS,...,TKL,FC,AW,BR,DW,AKS,CL,PUNCH,LMT,poss
Team,,,,,,,,,,,,,,,,,,,,,
Bayern Munich,-0.898850,2.973806,2.775593,1.611378,1.795907,2.673514,2.370940,2.120765,3.188202,0.987291,...,-2.081226,-1.795271,-0.716088,1.360479,-0.948188,0.407311,-3.083852,-1.680898,-0.596272,2.474804
Borussia Dortmund,1.390868,0.807589,0.118582,-1.716195,-0.305416,-0.128590,1.854117,2.366656,0.630185,-0.790167,...,-1.469764,-2.115407,-1.202503,-0.403610,-0.713429,-1.157638,-0.662533,0.096810,-0.596272,1.379674
RasenBallsport Leipzig,-0.117029,0.509248,0.807436,-0.068055,0.765069,0.934277,-0.233706,0.317571,1.470131,-0.663206,...,1.420784,0.653768,1.188352,1.945090,1.814138,1.581023,-0.321502,-1.147586,0.708380,-0.100780
Bayer Leverkusen,-0.271538,1.067016,1.341650,0.203158,1.102074,-0.901585,1.991632,0.710995,1.412862,0.701628,...,0.401681,-0.866877,-0.386314,2.796366,0.084750,1.189786,-0.543172,-0.614273,-1.248599,1.562196
Borussia M.Gladbach,2.184874,0.405477,0.273222,-0.600050,0.170355,0.257907,0.882059,-0.272566,0.343840,0.066822,...,-1.414177,-1.475135,-1.433344,-1.101041,-2.419341,-1.157638,-1.003564,-1.503127,0.056054,0.467065


In [80]:
df_scaled2.columns

Index(['ppda_coef', 'deep', 'S', 'CR', 'CCOP', 'CCSP', 'P', 'DR', 'IBS', 'OBS',
       'TBOX', 'FKCR', 'CRN', 'CRNCR', 'FKS', 'ALB', 'ATB', 'TOFF', 'BCS',
       'oppda_coef', 'deep_allowed', 'INT', 'BLK', 'TKL', 'FC', 'AW', 'BR',
       'DW', 'AKS', 'CL', 'PUNCH', 'LMT', 'poss'],
      dtype='object')

In [53]:
df_scaled3 = df_scaled2.reset_index()

In [54]:
df_scaled3.head()

,Team,ppda_coef,deep,S,CR,CCOP,CCSP,P,DR,IBS,...,TKL,FC,AW,BR,DW,AKS,CL,PUNCH,LMT,poss
0,Bayern Munich,-0.898850,2.973806,2.775593,1.611378,1.795907,2.673514,2.370940,2.120765,3.188202,...,-2.081226,-1.795271,-0.716088,1.360479,-0.948188,0.407311,-3.083852,-1.680898,-0.596272,2.474804
1,Borussia Dortmund,1.390868,0.807589,0.118582,-1.716195,-0.305416,-0.128590,1.854117,2.366656,0.630185,...,-1.469764,-2.115407,-1.202503,-0.403610,-0.713429,-1.157638,-0.662533,0.096810,-0.596272,1.379674
2,RasenBallsport Leipzig,-0.117029,0.509248,0.807436,-0.068055,0.765069,0.934277,-0.233706,0.317571,1.470131,...,1.420784,0.653768,1.188352,1.945090,1.814138,1.581023,-0.321502,-1.147586,0.708380,-0.100780
3,Bayer Leverkusen,-0.271538,1.067016,1.341650,0.203158,1.102074,-0.901585,1.991632,0.710995,1.412862,...,0.401681,-0.866877,-0.386314,2.796366,0.084750,1.189786,-0.543172,-0.614273,-1.248599,1.562196
4,Borussia M.Gladbach,2.184874,0.405477,0.273222,-0.600050,0.170355,0.257907,0.882059,-0.272566,0.343840,...,-1.414177,-1.475135,-1.433344,-1.101041,-2.419341,-1.157638,-1.003564,-1.503127,0.056054,0.467065


In [26]:
df1.Home.unique()

array(['Manchester Utd', 'Newcastle Utd', 'Huddersfield', 'Fulham',
       'Watford', 'Bournemouth', 'Wolves', 'Southampton', 'Liverpool',
       'Arsenal', 'Cardiff City', 'West Ham', 'Everton', 'Leicester City',
       'Tottenham', 'Chelsea', 'Burnley', 'Manchester City', 'Brighton',
       'Crystal Palace', 'Marseille', 'Nantes', 'Montpellier', 'Angers',
       'Saint-Étienne', 'Nice', 'Lille', 'Lyon', 'Bordeaux', 'Paris S-G',
       'Reims', 'Guingamp', 'Dijon', 'Rennes', 'Caen', 'Amiens', 'Monaco',
       'Strasbourg', 'Toulouse', 'Nîmes', 'Bayern Munich',
       'Werder Bremen', 'Freiburg', 'Wolfsburg', 'Hertha BSC',
       'Düsseldorf', "M'gladbach", 'Mainz 05', 'Dortmund', 'Hannover 96',
       'Augsburg', 'Hoffenheim', 'Bayer', 'Eint Frankfurt', 'Nürnberg',
       'Stuttgart', 'RB Leipzig', 'Schalke 04', 'Chievo', 'Lazio',
       'Torino', 'Empoli', 'Bologna', 'Parma', 'Sassuolo', 'Atalanta',
       'Juventus', 'Napoli', 'SPAL', 'Inter', 'Cagliari', 'Udinese',
       'Fiorenti

In [28]:
team_dict = {'Manchester Utd': 'Manchester United', 'Newcastle Utd': 'Newcastle United', 'Wolves': 
             'Wolverhampton Wanderers', 'Cardiff City': 'Cardiff', 'Leicester City': 'Leicester', 'Saint-Étienne': 
             'Saint-Etienne', 'Paris S-G': 'Paris Saint Germain', 'Nîmes': 'Nimes', 'Hertha BSC': 'Hertha Berlin', 
             'Düsseldorf': 'Fortuna Duesseldorf', "M'gladbach": 'Borussia M.Gladbach', 'Dortmund':
             'Borussia Dortmund', 'Bayer': 'Bayer Leverkusen', 'Eint Frankfurt': 'Eintracht Frankfurt',
             'Nürnberg': 'Nuernberg', 'Stuttgart': 'VfB Stuttgart', 'RB Leipzig': 'RasenBallsport Leipzig',
             'Parma': 'Parma Calcio 1913', 'SPAL': 'SPAL 2013', 'Milan': 'AC Milan', 'Betis': 'Real Betis',
             'Athletic Bilbao': 'Athletic Club', 'Leganés': 'Leganes', 'Alavés': 'Alaves', 'Atlético Madrid':
             'Atletico Madrid', 'Valladolid': 'Real Valladolid', 'Huesca': 'SD Huesca'}

In [20]:
df1 = pd.read_csv('df_xg.csv', index_col=0)

In [21]:
df1.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester Utd,1.6,1.1,Leicester City,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle Utd,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [46]:
# df2 = pd.read_csv('xg_df.csv', index_col=0)

In [43]:
# df2 = df2.set_index('index')

In [44]:
# df2.head()

,xg_home,xg_away,xg_total,xga_home,xga_away,xga_total,xg_dif
index,,,,,,,
Manchester United,31.4,28.1,59.5,20.3,28.0,48.3,11.2
Newcastle United,22.1,16.7,38.8,23.7,29.7,53.4,-14.6
Huddersfield,15.9,14.2,30.1,30.1,32.2,62.3,-32.2
Fulham,22.7,16.7,39.4,27.7,37.8,65.5,-26.1
Watford,23.1,24.8,47.9,27.5,27.6,55.1,-7.2


In [45]:
# df2['index'] = df2['index'].map(team_dict).fillna(df2['index'])

In [47]:
df1['Home'] = df1['Home'].map(team_dict).fillna(df1['Home'])

In [48]:
df1['Away'] = df1['Away'].map(team_dict).fillna(df1['Away'])

In [49]:
df1.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester United,1.6,1.1,Leicester,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle United,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [50]:
cols_to_drop = ['Wk', 'Day', 'Date', 'Attendance', 'Venue', 'Referee', 'league']

In [51]:
df1 = df1.drop(columns=cols_to_drop)

In [58]:
df1.shape

(1824, 6)

In [55]:
df_merged1 = pd.merge(df1, df_scaled3, left_on='Home', right_on='Team')

In [59]:
df_merged2 = pd.merge(df_merged1, df_scaled3, left_on='Away', right_on='Team')

In [68]:
df_merged2.Away.value_counts()

Cagliari                  19
Huddersfield              19
Real Madrid               19
Sevilla                   19
Levante                   19
                          ..
Nuernberg                 17
Augsburg                  17
Hertha Berlin             17
RasenBallsport Leipzig    17
Eintracht Frankfurt       17
Name: Away, Length: 98, dtype: int64

In [71]:
#df_merged2= df_merged2.drop(columns=['Team_x', 'Team_y'])

In [210]:
col_list1 = df_merged2.columns[73:107]

In [219]:
df_merged2.columns[1:3]

Index(['xG', 'xG.1'], dtype='object')

In [221]:
col_list2 = df_merged2.columns[6:72]

In [224]:
col_list2 = col_list2.append(df_merged2.columns[1:3])

In [232]:
col_list1

Index(['xG_diff', 'ppda_diff', 'deep_diff', 'S_diff', 'CR_diff', 'CCOP_diff',
       'CCSP_diff', 'P_diff', 'DR_diff', 'IBS_diff', 'OBS_diff', 'TBOX_diff',
       'FKCR_diff', 'CRN_diff', 'CRNCR_diff', 'FKS_diff', 'ALB_diff',
       'ATB_diff', 'TOFF_diff', 'BCS_diff', 'oppda_diff', 'deep_allowed_diff',
       'INT_diff', 'BLK_diff', 'TKL_diff', 'FC_diff', 'AW_diff', 'BR_diff',
       'DW_diff', 'AKS_diff', 'CL_diff', 'PUNCH_diff', 'LMT_diff',
       'poss_diff'],
      dtype='object')

In [81]:
df_merged2['xG_diff'] = df_merged2['xG'] - df_merged2['xG.1']
df_merged2['ppda_diff'] = df_merged2['ppda_coef_x'] - df_merged2['ppda_coef_y']
df_merged2['deep_diff'] = df_merged2['deep_x'] - df_merged2['deep_y']
df_merged2['S_diff'] = df_merged2['S_x'] - df_merged2['S_y']
df_merged2['CR_diff'] = df_merged2['CR_x'] - df_merged2['CR_y']
df_merged2['CCOP_diff'] = df_merged2['CCOP_x'] - df_merged2['CCOP_y']
df_merged2['CCSP_diff'] = df_merged2['CCSP_x'] - df_merged2['CCSP_y']
df_merged2['P_diff'] = df_merged2['P_x'] - df_merged2['P_y']
df_merged2['DR_diff'] = df_merged2['DR_x'] - df_merged2['DR_y']
df_merged2['IBS_diff'] = df_merged2['IBS_x'] - df_merged2['IBS_y']
df_merged2['OBS_diff'] = df_merged2['OBS_x'] - df_merged2['OBS_y']
df_merged2['TBOX_diff'] = df_merged2['TBOX_x'] - df_merged2['TBOX_y']
df_merged2['FKCR_diff'] = df_merged2['FKCR_x'] - df_merged2['FKCR_y']
df_merged2['CRN_diff'] = df_merged2['CRN_x'] - df_merged2['CRN_y']
df_merged2['CRNCR_diff'] = df_merged2['CRNCR_x'] - df_merged2['CRNCR_y']
df_merged2['FKS_diff'] = df_merged2['FKS_x'] - df_merged2['FKS_y']
df_merged2['ALB_diff'] = df_merged2['ALB_x'] - df_merged2['ALB_y']
df_merged2['ATB_diff'] = df_merged2['ATB_x'] - df_merged2['ATB_y']
df_merged2['TOFF_diff'] = df_merged2['TOFF_x'] - df_merged2['TOFF_y']
df_merged2['BCS_diff'] = df_merged2['BCS_x'] - df_merged2['BCS_y']
df_merged2['oppda_diff'] = df_merged2['oppda_coef_x'] - df_merged2['oppda_coef_y']
df_merged2['deep_allowed_diff'] = df_merged2['deep_allowed_x'] - df_merged2['deep_allowed_y']
df_merged2['INT_diff'] = df_merged2['INT_x'] - df_merged2['INT_y']
df_merged2['BLK_diff'] = df_merged2['BLK_x'] - df_merged2['BLK_y']
df_merged2['TKL_diff'] = df_merged2['TKL_x'] - df_merged2['TKL_y']
df_merged2['FC_diff'] = df_merged2['FC_x'] - df_merged2['FC_y']
df_merged2['AW_diff'] = df_merged2['AW_x'] - df_merged2['AW_y']
df_merged2['BR_diff'] = df_merged2['BR_x'] - df_merged2['BR_y']
df_merged2['DW_diff'] = df_merged2['DW_x'] - df_merged2['DW_y']
df_merged2['AKS_diff'] = df_merged2['AKS_x'] - df_merged2['AKS_y']
df_merged2['CL_diff'] = df_merged2['CL_x'] - df_merged2['CL_y']
df_merged2['PUNCH_diff'] = df_merged2['PUNCH_x'] - df_merged2['PUNCH_y']
df_merged2['LMT_diff'] = df_merged2['LMT_x'] - df_merged2['LMT_y']
df_merged2['poss_diff'] = df_merged2['poss_x'] - df_merged2['poss_y']

In [157]:
df_merged2['score'] = df_merged2['home_goals'] - df_merged2['away_goals']
results = df_merged2.drop(['home_goals', 'away_goals', 'Home', 'Away'], 1)
results['winner'] = None
results['winner'][results.score > 0] = 1
results['winner'][results.score < 0] = 2
results['winner'][results.score == 0] = 0
#results = results[results.winner != 0]

In [158]:
results.head(20)

,xG,xG.1,ppda_coef_x,deep_x,S_x,CR_x,CCOP_x,CCSP_x,P_x,DR_x,...,FC_diff,AW_diff,BR_diff,DW_diff,AKS_diff,CL_diff,PUNCH_diff,LMT_diff,poss_diff,winner
0,1.6,1.1,-0.409202,0.963928,0.436742,-0.539109,0.518836,-1.333851,0.650128,-0.152640,...,1.174393,-0.892557,-0.945202,-1.848415,-0.350054,-1.235788,-0.477174,-1.167321,0.507005,1
1,0.3,1.4,0.853560,-0.602876,-0.569533,0.142217,-0.527652,-0.555771,-1.089636,-1.428681,...,1.031175,-0.103271,0.036707,-1.288289,1.225191,1.334956,0.000000,-0.583660,-1.602135,2
2,1.7,3.0,0.197249,-1.125143,-1.148141,0.683545,-1.006553,-0.037051,-0.439448,-1.487350,...,0.988209,0.309813,0.321185,-0.700157,0.175027,-2.196956,0.954348,-1.750981,-0.750367,2
3,1.7,1.8,1.492186,-0.684117,-0.468906,-0.791106,-0.013276,-0.037051,0.188290,0.331375,...,1.031175,-0.501602,0.229418,-1.519341,1.575245,0.190708,2.226813,-2.334641,-0.324483,0
4,0.8,1.0,0.083612,0.522902,-0.682739,-1.173769,-0.616337,-1.852571,-0.523710,-0.621989,...,1.174393,-0.331943,0.119297,-1.169262,4.025627,0.083912,-0.795290,1.750981,-0.689527,1
5,1.7,2.1,1.737714,0.580932,-0.569533,-0.781773,-0.687286,-0.123505,-0.322822,-0.592654,...,-0.157541,-0.752403,-1.238857,-2.058462,0.875136,0.602637,-1.113406,0.583660,-0.628686,1
6,1.5,1.3,1.248633,-0.533240,-0.179601,-0.688441,-0.190647,-0.296411,-0.167710,-0.035303,...,0.830668,0.022130,1.385685,-0.511115,-1.050163,-0.923027,-1.113406,-1.167321,-0.689527,1
7,0.6,0.6,-0.074104,-0.277909,-0.091552,-0.277779,-0.403492,0.481668,-0.872129,-0.680657,...,0.988209,-0.309813,-0.100944,-1.106248,0.700109,0.930655,0.795290,0.000000,-1.095130,2
8,0.5,0.8,-0.448657,2.333430,1.053086,0.235549,0.767156,-0.296411,1.865368,0.316708,...,-0.515587,-1.563819,0.532250,-2.443548,1.925300,-2.341894,0.318116,-1.167321,1.622415,0
9,2.9,0.7,-0.753123,1.973646,-0.305386,-0.847106,0.093146,-1.852571,1.097679,-0.431316,...,0.873634,-1.866255,-0.192711,-2.303517,1.400218,-1.388354,0.477174,-1.167321,1.074850,1


In [172]:
y = results['winner'].astype(int)
X = results[col_list1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

In [181]:
# Instantiate XGBClassifier
xgb = XGBClassifier()


In [182]:
# Fit XGBClassifier
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [183]:
# Predict on training and test sets
training_preds = xgb.predict(X_train)
test_preds = xgb.predict(X_test)

In [184]:
# Accuracy of training and test sets
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

In [185]:
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

Training Accuracy: 77.19%
Validation accuracy: 55.48%


In [186]:
lr = LogisticRegression()

In [187]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [188]:
training_preds2 = lr.predict(X_train)
test_preds2 = lr.predict(X_test)

In [189]:
training_accuracy2 = accuracy_score(y_train, training_preds2)
test_accuracy2 = accuracy_score(y_test, test_preds2)

In [191]:
print('Training Accuracy: {:.4}%'.format(training_accuracy2 * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy2 * 100))

Training Accuracy: 60.45%
Validation accuracy: 56.14%


In [192]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [193]:
training_preds3 = rf.predict(X_train)
test_preds3 = rf.predict(X_test)

In [194]:
training_accuracy3 = accuracy_score(y_train, training_preds3)
test_accuracy3 = accuracy_score(y_test, test_preds3)

In [195]:
print('Training Accuracy: {:.4}%'.format(training_accuracy3 * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy3 * 100))

Training Accuracy: 100.0%
Validation accuracy: 54.82%


In [196]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [197]:
training_preds4 = svc.predict(X_train)
test_preds4 = svc.predict(X_test)

In [198]:
training_accuracy4 = accuracy_score(y_train, training_preds4)
test_accuracy4 = accuracy_score(y_test, test_preds4)

In [199]:
print('Training Accuracy: {:.4}%'.format(training_accuracy4 * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy4 * 100))

Training Accuracy: 60.31%
Validation accuracy: 55.7%


In [200]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [201]:
training_preds5 = knn.predict(X_train)
test_preds5 = knn.predict(X_test)

In [202]:
training_accuracy5 = accuracy_score(y_train, training_preds5)
test_accuracy5 = accuracy_score(y_test, test_preds5)

In [203]:
print('Training Accuracy: {:.4}%'.format(training_accuracy5 * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy5 * 100))

Training Accuracy: 62.43%
Validation accuracy: 42.11%


In [302]:
rf_f1 = f1_score(y_test, test_preds3, average='macro')

In [303]:
rf_f1

0.4688747944561899

In [300]:
svc_f1 = f1_score(y_test, test_preds4, average='macro')

In [301]:
svc_f1

0.48577626963399884

In [307]:
svc.get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [308]:
param_grid = {
    'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']
}

In [309]:
grid_svc = GridSearchCV(svc, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_svc.fit(X_train, y_train)

best_parameters = grid_svc.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_svc.predict(X_train)
test_preds = grid_svc.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

Grid Search found the following optimal parameters: 
C: 100
gamma: 0.0001
kernel: 'rbf'

Training Accuracy: 60.31%
Validation accuracy: 55.04%


In [231]:
df_merged2.columns

Index(['Home', 'xG', 'xG.1', 'Away', 'home_goals', 'away_goals', 'ppda_coef_x',
       'deep_x', 'S_x', 'CR_x',
       ...
       'TKL_diff', 'FC_diff', 'AW_diff', 'BR_diff', 'DW_diff', 'AKS_diff',
       'CL_diff', 'PUNCH_diff', 'LMT_diff', 'poss_diff'],
      dtype='object', length=107)

In [230]:
col_list2

Index(['ppda_coef_x', 'deep_x', 'S_x', 'CR_x', 'CCOP_x', 'CCSP_x', 'P_x',
       'DR_x', 'IBS_x', 'OBS_x', 'TBOX_x', 'FKCR_x', 'CRN_x', 'CRNCR_x',
       'FKS_x', 'ALB_x', 'ATB_x', 'TOFF_x', 'BCS_x', 'oppda_coef_x',
       'deep_allowed_x', 'INT_x', 'BLK_x', 'TKL_x', 'FC_x', 'AW_x', 'BR_x',
       'DW_x', 'AKS_x', 'CL_x', 'PUNCH_x', 'LMT_x', 'poss_x', 'ppda_coef_y',
       'deep_y', 'S_y', 'CR_y', 'CCOP_y', 'CCSP_y', 'P_y', 'DR_y', 'IBS_y',
       'OBS_y', 'TBOX_y', 'FKCR_y', 'CRN_y', 'CRNCR_y', 'FKS_y', 'ALB_y',
       'ATB_y', 'TOFF_y', 'BCS_y', 'oppda_coef_y', 'deep_allowed_y', 'INT_y',
       'BLK_y', 'TKL_y', 'FC_y', 'AW_y', 'BR_y', 'DW_y', 'AKS_y', 'CL_y',
       'PUNCH_y', 'LMT_y', 'poss_y', 'xG', 'xG.1'],
      dtype='object')

In [290]:
(df_merged2[df_merged2.Away == 'Arsenal']['xG.1'].mean() + df_merged2[df_merged2.Home == 'Arsenal']['xG'].mean())/2

1.4842105263157892

In [281]:
### with home-field advantage
def match1(df, team1, team2, model):
    
    match = pd.DataFrame(columns=col_list2, index=[0])
    
    match['ppda_coef_x'] = df[df.Home == team1]['ppda_coef_x'].iloc[0]
    match['deep_x'] = df[df.Home == team1]['deep_x'].iloc[0]
    match['S_x'] = df[df.Home == team1]['S_x'].iloc[0]
    match['CR_x'] = df[df.Home == team1]['CR_x'].iloc[0]
    match['CCOP_x'] = df[df.Home == team1]['CCOP_x'].iloc[0]
    match['CCSP_x'] = df[df.Home == team1]['CCSP_x'].iloc[0]
    match['P_x'] = df[df.Home == team1]['P_x'].iloc[0]
    match['DR_x'] = df[df.Home == team1]['DR_x'].iloc[0]
    match['IBS_x'] = df[df.Home == team1]['IBS_x'].iloc[0]
    match['OBS_x'] = df[df.Home == team1]['OBS_x'].iloc[0]
    match['TBOX_x'] = df[df.Home == team1]['TBOX_x'].iloc[0]
    match['FKCR_x'] = df[df.Home == team1]['FKCR_x'].iloc[0]
    match['CRN_x'] = df[df.Home == team1]['CRN_x'].iloc[0]
    match['CRNCR_x'] = df[df.Home == team1]['CRNCR_x'].iloc[0]
    match['FKS_x'] = df[df.Home == team1]['FKS_x'].iloc[0]
    match['ALB_x'] = df[df.Home == team1]['ALB_x'].iloc[0]
    match['ATB_x'] = df[df.Home == team1]['ATB_x'].iloc[0]
    match['TOFF_x'] = df[df.Home == team1]['TOFF_x'].iloc[0]
    match['BCS_x'] = df[df.Home == team1]['BCS_x'].iloc[0]
    match['oppda_coef_x'] = df[df.Home == team1]['oppda_coef_x'].iloc[0]
    match['deep_allowed_x'] = df[df.Home == team1]['deep_allowed_x'].iloc[0]
    match['INT_x'] = df[df.Home == team1]['INT_x'].iloc[0]
    match['BLK_x'] = df[df.Home == team1]['BLK_x'].iloc[0]
    match['TKL_x'] = df[df.Home == team1]['TKL_x'].iloc[0]
    match['FC_x'] = df[df.Home == team1]['FC_x'].iloc[0]
    match['AW_x'] = df[df.Home == team1]['AW_x'].iloc[0]
    match['BR_x'] = df[df.Home == team1]['BR_x'].iloc[0]
    match['DW_x'] = df[df.Home == team1]['DW_x'].iloc[0]
    match['AKS_x'] = df[df.Home == team1]['AKS_x'].iloc[0]
    match['CL_x'] = df[df.Home == team1]['CL_x'].iloc[0]
    match['PUNCH_x'] = df[df.Home == team1]['PUNCH_x'].iloc[0]
    match['LMT_x'] = df[df.Home == team1]['LMT_x'].iloc[0]
    match['poss_x'] = df[df.Home == team1]['poss_x'].iloc[0]
    match['xG'] = df[df.Home == team1]['xG'].mean()
    
    match['ppda_coef_y'] = df[df.Away == team2]['ppda_coef_y'].iloc[0]
    match['deep_y'] = df[df.Away == team2]['deep_y'].iloc[0]
    match['S_y'] = df[df.Away == team2]['S_y'].iloc[0]
    match['CR_y'] = df[df.Away == team2]['CR_y'].iloc[0]
    match['CCOP_y'] = df[df.Away == team2]['CCOP_y'].iloc[0]
    match['CCSP_y'] = df[df.Away == team2]['CCSP_y'].iloc[0]
    match['P_y'] = df[df.Away == team2]['P_y'].iloc[0]
    match['DR_y'] = df[df.Away == team2]['DR_y'].iloc[0]
    match['IBS_y'] = df[df.Away == team2]['IBS_y'].iloc[0]
    match['OBS_y'] = df[df.Away == team2]['OBS_y'].iloc[0]
    match['TBOX_y'] = df[df.Away == team2]['TBOX_y'].iloc[0]
    match['FKCR_y'] = df[df.Away == team2]['FKCR_y'].iloc[0]
    match['CRN_y'] = df[df.Away == team2]['CRN_y'].iloc[0]
    match['CRNCR_y'] = df[df.Away == team2]['CRNCR_y'].iloc[0]
    match['FKS_y'] = df[df.Away == team2]['FKS_y'].iloc[0]
    match['ALB_y'] = df[df.Away == team2]['ALB_y'].iloc[0]
    match['ATB_y'] = df[df.Away == team2]['ATB_y'].iloc[0]
    match['TOFF_y'] = df[df.Away == team2]['TOFF_y'].iloc[0]
    match['BCS_y'] = df[df.Away == team2]['BCS_y'].iloc[0]
    match['oppda_coef_y'] = df[df.Away == team2]['oppda_coef_y'].iloc[0]
    match['deep_allowed_y'] = df[df.Away == team2]['deep_allowed_y'].iloc[0]
    match['INT_y'] = df[df.Away == team2]['INT_y'].iloc[0]
    match['BLK_y'] = df[df.Away == team2]['BLK_y'].iloc[0]
    match['TKL_y'] = df[df.Away == team2]['TKL_y'].iloc[0]
    match['FC_y'] = df[df.Away == team2]['FC_x'].iloc[0]
    match['AW_y'] = df[df.Away == team2]['AW_y'].iloc[0]
    match['BR_y'] = df[df.Away == team2]['BR_y'].iloc[0]
    match['DW_y'] = df[df.Away == team2]['DW_y'].iloc[0]
    match['AKS_y'] = df[df.Away == team2]['AKS_y'].iloc[0]
    match['CL_y'] = df[df.Away == team2]['CL_y'].iloc[0]
    match['PUNCH_y'] = df[df.Away == team2]['PUNCH_y'].iloc[0]
    match['LMT_y'] = df[df.Away == team2]['LMT_y'].iloc[0]
    match['poss_y'] = df[df.Away == team2]['poss_y'].iloc[0]
    match['xG.1'] = df[df.Away == team2]['xG.1'].mean()
    
    match['xG_diff'] = match['xG'] - match['xG.1']
    match['ppda_diff'] = match['ppda_coef_x'] - match['ppda_coef_y']
    match['deep_diff'] = match['deep_x'] - match['deep_y']
    match['S_diff'] = match['S_x'] - match['S_y']
    match['CR_diff'] = match['CR_x'] - match['CR_y']
    match['CCOP_diff'] = match['CCOP_x'] - match['CCOP_y']
    match['CCSP_diff'] = match['CCSP_x'] - match['CCSP_y']
    match['P_diff'] = match['P_x'] - match['P_y']
    match['DR_diff'] = match['DR_x'] - match['DR_y']
    match['IBS_diff'] = match['IBS_x'] - match['IBS_y']
    match['OBS_diff'] = match['OBS_x'] - match['OBS_y']
    match['TBOX_diff'] = match['TBOX_x'] - match['TBOX_y']
    match['FKCR_diff'] = match['FKCR_x'] - match['FKCR_y']
    match['CRN_diff'] = match['CRN_x'] - match['CRN_y']
    match['CRNCR_diff'] = match['CRNCR_x'] - match['CRNCR_y']
    match['FKS_diff'] = match['FKS_x'] - match['FKS_y']
    match['ALB_diff'] = match['ALB_x'] - match['ALB_y']
    match['ATB_diff'] = match['ATB_x'] - match['ATB_y']
    match['TOFF_diff'] = match['TOFF_x'] - match['TOFF_y']
    match['BCS_diff'] = match['BCS_x'] - match['BCS_y']
    match['oppda_diff'] = match['oppda_coef_x'] - match['oppda_coef_y']
    match['deep_allowed_diff'] = match['deep_allowed_x'] - match['deep_allowed_y']
    match['INT_diff'] = match['INT_x'] - match['INT_y']
    match['BLK_diff'] = match['BLK_x'] - match['BLK_y']
    match['TKL_diff'] = match['TKL_x'] - match['TKL_y']
    match['FC_diff'] = match['FC_x'] - match['FC_y']
    match['AW_diff'] = match['AW_x'] - match['AW_y']
    match['BR_diff'] = match['BR_x'] - match['BR_y']
    match['DW_diff'] = match['DW_x'] - match['DW_y']
    match['AKS_diff'] = match['AKS_x'] - match['AKS_y']
    match['CL_diff'] = match['CL_x'] - match['CL_y']
    match['PUNCH_diff'] = match['PUNCH_x'] - match['PUNCH_y']
    match['LMT_diff'] = match['LMT_x'] - match['LMT_y']
    match['poss_diff'] = match['poss_x'] - match['poss_y']
    
    match = match[col_list1]
    
    match_array = match.values
    
    prediction = model.predict(match_array)
    
    winner = None
    
    if prediction == 1:
        winner = team1
    elif prediction == 2:
        winner = team2
    else:
        winner = 'Tie'
    
    return winner

In [298]:
match1(df_merged2, 'Arsenal', 'Liverpool', rf)

'Tie'

In [291]:
### no home-field advantage
def match2(df, team1, team2, model):
    
    match = pd.DataFrame(columns=col_list2, index=[0])
    
    match['ppda_coef_x'] = df[df.Home == team1]['ppda_coef_x'].iloc[0]
    match['deep_x'] = df[df.Home == team1]['deep_x'].iloc[0]
    match['S_x'] = df[df.Home == team1]['S_x'].iloc[0]
    match['CR_x'] = df[df.Home == team1]['CR_x'].iloc[0]
    match['CCOP_x'] = df[df.Home == team1]['CCOP_x'].iloc[0]
    match['CCSP_x'] = df[df.Home == team1]['CCSP_x'].iloc[0]
    match['P_x'] = df[df.Home == team1]['P_x'].iloc[0]
    match['DR_x'] = df[df.Home == team1]['DR_x'].iloc[0]
    match['IBS_x'] = df[df.Home == team1]['IBS_x'].iloc[0]
    match['OBS_x'] = df[df.Home == team1]['OBS_x'].iloc[0]
    match['TBOX_x'] = df[df.Home == team1]['TBOX_x'].iloc[0]
    match['FKCR_x'] = df[df.Home == team1]['FKCR_x'].iloc[0]
    match['CRN_x'] = df[df.Home == team1]['CRN_x'].iloc[0]
    match['CRNCR_x'] = df[df.Home == team1]['CRNCR_x'].iloc[0]
    match['FKS_x'] = df[df.Home == team1]['FKS_x'].iloc[0]
    match['ALB_x'] = df[df.Home == team1]['ALB_x'].iloc[0]
    match['ATB_x'] = df[df.Home == team1]['ATB_x'].iloc[0]
    match['TOFF_x'] = df[df.Home == team1]['TOFF_x'].iloc[0]
    match['BCS_x'] = df[df.Home == team1]['BCS_x'].iloc[0]
    match['oppda_coef_x'] = df[df.Home == team1]['oppda_coef_x'].iloc[0]
    match['deep_allowed_x'] = df[df.Home == team1]['deep_allowed_x'].iloc[0]
    match['INT_x'] = df[df.Home == team1]['INT_x'].iloc[0]
    match['BLK_x'] = df[df.Home == team1]['BLK_x'].iloc[0]
    match['TKL_x'] = df[df.Home == team1]['TKL_x'].iloc[0]
    match['FC_x'] = df[df.Home == team1]['FC_x'].iloc[0]
    match['AW_x'] = df[df.Home == team1]['AW_x'].iloc[0]
    match['BR_x'] = df[df.Home == team1]['BR_x'].iloc[0]
    match['DW_x'] = df[df.Home == team1]['DW_x'].iloc[0]
    match['AKS_x'] = df[df.Home == team1]['AKS_x'].iloc[0]
    match['CL_x'] = df[df.Home == team1]['CL_x'].iloc[0]
    match['PUNCH_x'] = df[df.Home == team1]['PUNCH_x'].iloc[0]
    match['LMT_x'] = df[df.Home == team1]['LMT_x'].iloc[0]
    match['poss_x'] = df[df.Home == team1]['poss_x'].iloc[0]
    match['xG'] = df[df.Home == team1]['xG'].iloc[0]
    
    match['ppda_coef_y'] = df[df.Away == team2]['ppda_coef_y'].iloc[0]
    match['deep_y'] = df[df.Away == team2]['deep_y'].iloc[0]
    match['S_y'] = df[df.Away == team2]['S_y'].iloc[0]
    match['CR_y'] = df[df.Away == team2]['CR_y'].iloc[0]
    match['CCOP_y'] = df[df.Away == team2]['CCOP_y'].iloc[0]
    match['CCSP_y'] = df[df.Away == team2]['CCSP_y'].iloc[0]
    match['P_y'] = df[df.Away == team2]['P_y'].iloc[0]
    match['DR_y'] = df[df.Away == team2]['DR_y'].iloc[0]
    match['IBS_y'] = df[df.Away == team2]['IBS_y'].iloc[0]
    match['OBS_y'] = df[df.Away == team2]['OBS_y'].iloc[0]
    match['TBOX_y'] = df[df.Away == team2]['TBOX_y'].iloc[0]
    match['FKCR_y'] = df[df.Away == team2]['FKCR_y'].iloc[0]
    match['CRN_y'] = df[df.Away == team2]['CRN_y'].iloc[0]
    match['CRNCR_y'] = df[df.Away == team2]['CRNCR_y'].iloc[0]
    match['FKS_y'] = df[df.Away == team2]['FKS_y'].iloc[0]
    match['ALB_y'] = df[df.Away == team2]['ALB_y'].iloc[0]
    match['ATB_y'] = df[df.Away == team2]['ATB_y'].iloc[0]
    match['TOFF_y'] = df[df.Away == team2]['TOFF_y'].iloc[0]
    match['BCS_y'] = df[df.Away == team2]['BCS_y'].iloc[0]
    match['oppda_coef_y'] = df[df.Away == team2]['oppda_coef_y'].iloc[0]
    match['deep_allowed_y'] = df[df.Away == team2]['deep_allowed_y'].iloc[0]
    match['INT_y'] = df[df.Away == team2]['INT_y'].iloc[0]
    match['BLK_y'] = df[df.Away == team2]['BLK_y'].iloc[0]
    match['TKL_y'] = df[df.Away == team2]['TKL_y'].iloc[0]
    match['FC_y'] = df[df.Away == team2]['FC_x'].iloc[0]
    match['AW_y'] = df[df.Away == team2]['AW_y'].iloc[0]
    match['BR_y'] = df[df.Away == team2]['BR_y'].iloc[0]
    match['DW_y'] = df[df.Away == team2]['DW_y'].iloc[0]
    match['AKS_y'] = df[df.Away == team2]['AKS_y'].iloc[0]
    match['CL_y'] = df[df.Away == team2]['CL_y'].iloc[0]
    match['PUNCH_y'] = df[df.Away == team2]['PUNCH_y'].iloc[0]
    match['LMT_y'] = df[df.Away == team2]['LMT_y'].iloc[0]
    match['poss_y'] = df[df.Away == team2]['poss_y'].iloc[0]
    match['xG.1'] = (df[df.Home == team2]['xG'].mean() + df[df.Away == team2]['xG.1'].mean())/2
    
    match['xG_diff'] = match['xG'] - match['xG.1']
    match['ppda_diff'] = match['ppda_coef_x'] - match['ppda_coef_y']
    match['deep_diff'] = match['deep_x'] - match['deep_y']
    match['S_diff'] = match['S_x'] - match['S_y']
    match['CR_diff'] = match['CR_x'] - match['CR_y']
    match['CCOP_diff'] = match['CCOP_x'] - match['CCOP_y']
    match['CCSP_diff'] = match['CCSP_x'] - match['CCSP_y']
    match['P_diff'] = match['P_x'] - match['P_y']
    match['DR_diff'] = match['DR_x'] - match['DR_y']
    match['IBS_diff'] = match['IBS_x'] - match['IBS_y']
    match['OBS_diff'] = match['OBS_x'] - match['OBS_y']
    match['TBOX_diff'] = match['TBOX_x'] - match['TBOX_y']
    match['FKCR_diff'] = match['FKCR_x'] - match['FKCR_y']
    match['CRN_diff'] = match['CRN_x'] - match['CRN_y']
    match['CRNCR_diff'] = match['CRNCR_x'] - match['CRNCR_y']
    match['FKS_diff'] = match['FKS_x'] - match['FKS_y']
    match['ALB_diff'] = match['ALB_x'] - match['ALB_y']
    match['ATB_diff'] = match['ATB_x'] - match['ATB_y']
    match['TOFF_diff'] = match['TOFF_x'] - match['TOFF_y']
    match['BCS_diff'] = match['BCS_x'] - match['BCS_y']
    match['oppda_diff'] = match['oppda_coef_x'] - match['oppda_coef_y']
    match['deep_allowed_diff'] = match['deep_allowed_x'] - match['deep_allowed_y']
    match['INT_diff'] = match['INT_x'] - match['INT_y']
    match['BLK_diff'] = match['BLK_x'] - match['BLK_y']
    match['TKL_diff'] = match['TKL_x'] - match['TKL_y']
    match['FC_diff'] = match['FC_x'] - match['FC_y']
    match['AW_diff'] = match['AW_x'] - match['AW_y']
    match['BR_diff'] = match['BR_x'] - match['BR_y']
    match['DW_diff'] = match['DW_x'] - match['DW_y']
    match['AKS_diff'] = match['AKS_x'] - match['AKS_y']
    match['CL_diff'] = match['CL_x'] - match['CL_y']
    match['PUNCH_diff'] = match['PUNCH_x'] - match['PUNCH_y']
    match['LMT_diff'] = match['LMT_x'] - match['LMT_y']
    match['poss_diff'] = match['poss_x'] - match['poss_y']
    
    match = match[col_list1]
    
    match_array = match.values
    
    prediction = model.predict(match_array)
    
    winner = None
    
    if prediction == 1:
        winner = team1
    elif prediction == 2:
        winner = team2
    else:
        winner = 'Tie'
    
    return winner

In [299]:
match2(df_merged2, 'Watford', 'Liverpool', rf)

'Liverpool'